In [1]:
import numpy as np
import tensorflow as tf
import gym

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
%pip uninstall -y gym


Found existing installation: gym 0.26.2
Uninstalling gym-0.26.2:
  Successfully uninstalled gym-0.26.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install gymnasium


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/951.1 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/951.1 kB ? eta -:--:--
   ---------------------------------------- 951.1/951.1 kB 5.3 MB/s  0:00:00

   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ----------------

In [5]:
%pip install pygame

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   -------- ------------------------------- 2.4/10.6 MB 11.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/10.6 MB 6.0 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/10.6 MB 5.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.6 MB 5.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.0/10.6 MB 5.5 MB/s eta 0:00:01
   ------------------------- -------------- 6.8/10.6 MB 5.3 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.6 MB 5.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.6 MB 5.0 MB/s eta 0:00:01
   ----------------------------------- ---- 9.4/10.6 MB 4.9 MB/s eta 0:00:01
   -------------------------------------- - 10.2/10.6 MB 4.8 MB/s eta 0:00:01
   -------

In [4]:
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()


DependencyNotInstalled: pygame is not installed, run `pip install "gymnasium[classic-control]"`

In [6]:
%pip install "gymnasium[classic-control]"


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
